## Lendo os dados da Delta Table

No passo anterior você iniciou a leitura dos dados do Kafka para escrever nas Delta Tables, com o código abaixo é possível visualizar a evolução dos dados.
Na etapa anterior você utilizou o *Spark Streamings* para ler os itens em buckets de 1.000 e escrever no HDFS, agora vamos ver se os dados já foram todos escritos.

Lembrando, existem **15.000** clientes, **310.976** pedidos e **2.410.176** itens de pedido.

## Preparando o ambiente

O código abaixo adiciona a **raiz** do projeto, que contém códigos e dados necessários para o "Hands on".

In [6]:
root = '/home/bigdata/jupyterhub'

import sys
sys.path.append(root)

wd = '/delta'

O trecho de código abaixo prepara o ambiente, carregando códigos auxiliares e dados de configuração.O código disponível no pacote *commom.utils* na classe *DataframeUtils* contém vários métodos que facilitam a leitura e escrita dos dados do Postgres. A classe *DataframeUtils* também inicia uma instância do Apache Spark com o Delta Lake integrado ao Spark.

Já o arquivo *config.yaml* tem os dados de acesso ao Postgres e Kafka.

In [7]:
import yaml

from common.utils import DataframeUtils

config = yaml.safe_load(open('../config.yaml'))
dfu = DataframeUtils(config)

O trecho abaixo cria o Dataframe Spark de pedidos apontando para o diretório do HDFS onde o Spark Streaming está escrevendo.
Lembre-se, essa operação no Spark é *lazy*, portanto o código abaixo cria apenas o *link* entre o Spark e os dados no HDFS.

In [8]:
pedidos = dfu. \
  spark(). \
  read. \
  format('delta'). \
  load(f'{wd}/data/pedidos-bronze')

O trecho abaixo obtém a quantidade de itens no Dataframe, execute-o algumas vezes em intervalos de alguns segundos e observe o valor sendo incrementado.

Veja que não é necessário recriar o Dataframe para que o valor seja atualizado, o Spark já cuida dessa parte para você.
O valor final deverá ser **310.976**.

In [9]:
pedidos.count()

310976

O trecho abaixo cria o Dataframe de clientes exatamente da mesma forma que o de pedidos, alterando apenas o diretório no HDFS.

In [12]:
clientes = dfu. \
  spark(). \
  read. \
  format('delta'). \
  load(f'{wd}/data/clientes-bronze')

A quantidade de clientes é muito pequena, portanto quando você executar o trecho abaixo já terá finalizado a escrita dos **15.000** clientes.

In [13]:
clientes.count()

15000

O trecho abaixo cria uma *temp view* do Spark para que possamos executar uma consulta SQL.

In [14]:
clientes.createOrReplaceTempView('clientes_bronze')

A SQL abaixo é para verificar que não existem clientes repetidos.

Para isso realizamos um *group_by* na chave (key) e adicionamos um *having count > 1*.
O resultado abaixo deverá retornar uma lista vazia.

In [15]:
dfu.spark().sql("""
select key, count(value)
from clientes_bronze
group by 1
having count(value) > 1
""")

key,count(value)


# Exercício

Quer saber quantos itens de pedido foram todos escritos?
Adicione abaixo o código responsável por criar o Dataframe de itens de pedido e imprimir a quantidade de itens do Dataframe.

O diretório no HDFS é o **/delta/data/itens-bronze** e o count final deverá ser **2.410.176**.

In [10]:
# Complete o código para ler os dados de itens de pedido
df_itens = dfu. \
  spark(). \
  read. \
  format('delta'). \
  load(f'{wd}/data/itens-bronze')

In [11]:
# Adicione o código para imprimir a quantidade de itens do Dataframe
df_itens.count()

2410176